# Wk3: Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Question 1: Data Preparation
1. For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.


2. Start by creating a new Notebook for this assignment.
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

3. To create the above dataframe:

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
4. Submit a link to your Notebook on your Github repository. (10 marks)

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

The package is so popular that there is a plethora of tutorials and examples on how to use it. Here is a very good Youtube video on how to use the BeautifulSoup package: https://www.youtube.com/watch?v=ng2o98k983k

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

In [2]:
#1-2
import requests
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [39]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')

My_table = soup.find('table',{'class':'wikitable sortable'})

In [12]:
links = My_table.findAll('td')
Neighborhoods = []
for link in links:
    Neighborhoods.append(link.get('title'))

In [40]:
contents = My_table.findAll('td')
thedata = []
for content in contents:
    thedata.append(content.text)

Postcodes = []
Boroughs = []
Neighbourhoods = []

for data in thedata:
    if thedata.index(data) % 3 == 0:
        Postcodes.append(data)
    elif thedata.index(data) % 3 == 1:
        Boroughs.append(data)
    else:
        Neighbourhoods.append(data)
Neighbourhoods = [x[:-2] for x in Neighbourhoods]

In [70]:
import pandas as pd
df = pd.DataFrame()
df['Postcode']=Postcodes
df['Borough']=Boroughs
df['Neighborhood']=Neighbourhoods

In [71]:
#3
#drop all not assigned of Borough
df = df[df.Borough != 'Not assigned']

In [72]:
#replace not assigned neighbourhood with its borough
df['Neighborhood']= df['Neighborhood'].replace("Not assigne", df['Borough']) 

In [77]:
#combine neighborhoods within the same borough
df = df.groupby('Postcode', as_index=False).agg({'Neighborhood' : ', '.join, 'Borough' : 'first'})  
df = df[['Postcode', 'Borough', 'Neighborhood']]

In [79]:
print(df.shape)
df.head()

(103, 3)


,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Roug, Malver"
1,M1C,Scarborough,"Highland Cree, Rouge Hil, Port Unio"
2,M1E,Scarborough,"Guildwoo, Morningsid, West Hil"
3,M1G,Scarborough,Wobur
4,M1H,Scarborough,Cedarbra



## Question 2: Find/Add Coordinates
Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [88]:
#unable to import the module geocoder
co = pd.read_csv('http://cocl.us/Geospatial_data')
df = df.join(co.set_index('Postal Code'), on='Postcode')
df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Roug, Malver",43.806686,-79.194353
1,M1C,Scarborough,"Highland Cree, Rouge Hil, Port Unio",43.784535,-79.160497
2,M1E,Scarborough,"Guildwoo, Morningsid, West Hil",43.763573,-79.188711
3,M1G,Scarborough,Wobur,43.770992,-79.216917
4,M1H,Scarborough,Cedarbra,43.773136,-79.239476
